## Segmenting and Clustering Neighborhoods in Toronto

## Ans 1

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Installing required libraries

In [2]:
!conda install lxml --yes


!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim


!conda install -c conda-forge folium=0.5.0 --yes
import folium


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be UPDATED:

    openssl: 1.0.2r-h14c3975_0 conda-forge --> 1.0.2s-h7b6447c_0

The following packages will be DOWNGRADED:

    certifi: 2019.6.16-py36_1  conda-forge --> 2019.6.16-py36_0 

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be UPDATED:


## URL to get postal codes

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

## Using Pandas to parse the table on the above web page

In [4]:
tables = pd.read_html(url, header =0)


In [5]:
df = tables[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Removing rows without borough

In [6]:
df2 = df[df.Borough != 'Not assigned']

In [7]:
df3 = df2

In [8]:
df4 = df3.pivot_table( index = ['Postcode','Borough'], values = 'Neighbourhood', aggfunc=','.join )
df4.reset_index(inplace = True)
df4.head()
df5 = df4

## Getting neighbourhood value same as borough if neighbourhood value not assigned

In [9]:
for count, i in enumerate(df5['Neighbourhood']):
    if(i=="Not assigned"):
        df5.loc[count]["Neighbourhood"] = df5.loc[count]["Borough"]

In [10]:
df5.shape

(103, 3)

## Ans2

## url for coordinates

In [11]:
urlcood = 'https://cocl.us/Geospatial_data'

In [12]:
dfCood = pd.read_csv(urlcood)

In [13]:
dfCood.shape

(103, 3)

In [14]:
dfCood.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the tables

In [15]:
result = pd.merge(df5, dfCood, left_on = ['Postcode'], right_on=['Postal Code'], how = 'left')

In [16]:
result.head()


,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## Removing extra column

In [17]:
result.drop(columns= ['Postal Code'])
result.shape

(103, 6)

## Ans3

In [18]:
address = 'Toronto, Canada'


## Getting coordinates for the address

In [19]:
geolocator = Nominatim(user_agent = 'city_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [20]:
print("Geographical coordinates of Toronto are {}, {}".format(latitude,longitude))

Geographical coordinates of Toronto are 43.653963, -79.387207


## Map for toronto

In [40]:
map_toronto = folium.Map(location = [latitude,longitude], zoom_start =10)
location_df = result
for lat, lng, borough, neighbourhood in zip(location_df['Latitude'],location_df['Longitude'],location_df['Borough'],location_df['Neighbourhood']):
    label = '{},{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat,lng],
    radius =5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = 'red',
    fill_opacity =0.7,
    parse_html=False).add_to(map_toronto)

In [41]:
map_toronto

In [42]:
map_toronto.save('map_toronto.html')

## Check the link for the map

## https://github.com/sharmashubham7/Coursera_Capstone/blob/master/toronto_map.png

In [23]:
'''

FOURESQUARE CREDENTIALS

'''






## Exploring Neighbourhoods in toronto

In [24]:
LIMIT =100
def explore(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
toronto_venues = explore(names = result["Neighbourhood"],
                        latitudes = result["Latitude"],
                        longitudes = result["Longitude"])

In [26]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,55,55,55,55,55,55


## One hot encoding for venue categories

In [27]:
toronto_onehot_encoding = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot_encoding['Neighbourhood'] = toronto_venues['Neighbourhood']
fixed_columns = [toronto_onehot_encoding.columns[-1]] + list(toronto_onehot_encoding.columns[:-1])
toronto_onehot_encoding = toronto_onehot_encoding[fixed_columns]

In [28]:
toronto_grouped = toronto_onehot_encoding.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.0,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
4,"Alderwood,Long Branch",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.043478,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000


## Using only select top venues for location

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

## Clustering locations into 7 clusters using KMeans

In [31]:
kclusters = 7
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([3, 3, 6, 3, 0, 3, 3, 3, 3, 3], dtype=int32)

In [32]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = result

In [33]:
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

## Checking for null values

In [34]:
toronto_merged.index[toronto_merged["Cluster Labels"].isnull() == True].tolist()

[16, 93]

## Dropping null values

In [35]:
toronto_merged = toronto_merged.dropna()

In [43]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       

## Toronto Map with clusters

In [46]:
map_clusters

In [47]:
map_clusters.save('map_clusters.html')

## Check the link for the map

## https://github.com/sharmashubham7/Coursera_Capstone/blob/master/cluster_map.png